In [ ]:
import arcpy
import os
from arcpy import env

### Workspace and geodatabase
proj_gdb = r"" # replace with your gdb
ws = arcpy.env.workspace = proj_gdb
arcpy.env.overwriteOutput = True

### Declare feature classes
hunting_fc = "HuntingAssessmentParcels"
roads_fc = "MassGIS_Roads_2024_Western_MA_Clip"
towns_fc = "MassGIS_Towns_Western_MA_Clip"
structs_fc = "MassGISStructures2024_Western_MA_Clip"
hunting_fc_fields = ["Name", "Town"]

hunting_proximate_roads_lyr = "Hunting_proximate_roads_lyr"
hunting_proximate_structs_lyr = "Hunting_proximate_structs_lyr"

### Select roads and structures around hunting parcels - 2000 ft is adjustable
### This step reduces the size of the roads and structures layers, and so improves speed
def getSurroundingDataByLocation():
    select_distance = "2000 Feet"
    roads_lyr = 'roads_lyr'
    structs_lyr = 'structs_lyr'
    arcpy.management.MakeFeatureLayer(roads_fc, roads_lyr)
    arcpy.management.SelectLayerByLocation(roads_lyr, "WITHIN_A_DISTANCE", hunting_fc, select_distance)
    arcpy.management.CopyFeatures(roads_lyr, hunting_proximate_roads_lyr)
    
    arcpy.management.MakeFeatureLayer(structs_fc, structs_lyr)
    arcpy.management.SelectLayerByLocation(structs_lyr, "WITHIN_A_DISTANCE", hunting_fc, select_distance)
    arcpy.management.CopyFeatures(structs_lyr, hunting_proximate_structs_lyr)

getSurroundingDataByLocation()

### Declare reusable buffer variables
road_buf = hunting_proximate_roads_lyr + "_buffer"
struct_buf = hunting_proximate_structs_lyr + "_buffer"
all_buf = "All_Buffers"

### Create and merge buffers
def createBuffers():
    arcpy.analysis.Buffer(hunting_proximate_roads_lyr, road_buf, "150 Feet", "FULL", "ROUND", "ALL")
    arcpy.analysis.Buffer(hunting_proximate_structs_lyr, struct_buf, "500 Feet", "FULL", "ROUND", "ALL")
    arcpy.management.Merge([road_buf, struct_buf], all_buf)
    
createBuffers()

### Intersect buffers with hunting parcels to produce final output feature class
def intersectBuffersAndParcels():
    arcpy.analysis.Intersect([hunting_fc, all_buf], "All_Hunting_Buffer_Intersect", "NO_FID")
    
intersectBuffersAndParcels()